# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import cv2
import os
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
import joblib
import re

# Extracting GLCM features

In [3]:
# Function to compute GLCM features for six properties across four directions
def compute_glcm_features_4_directions(window, distances=[1], levels=256):
    # Convert to grayscale if the window is not already
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    
    # Ensure the window pixel intensity is in the range [0, levels-1]
    window = np.clip(window, 0, levels - 1).astype(np.uint8)
    
    # Define angles for the four directions: 0°, 45°, 90°, 135°
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    
    # Compute GLCM
    glcm = graycomatrix(window, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
    
    # Extract GLCM properties for six features
    features = {}
    features['contrast'] = np.mean(graycoprops(glcm, 'contrast'))
    features['dissimilarity'] = np.mean(graycoprops(glcm, 'dissimilarity'))
    features['homogeneity'] = np.mean(graycoprops(glcm, 'homogeneity'))
    features['ASM'] = np.mean(graycoprops(glcm, 'ASM'))  # Angular Second Moment (Energy)
    features['energy'] = np.mean(np.sqrt(graycoprops(glcm, 'ASM')))  # Energy is sqrt(ASM)
    features['correlation'] = np.mean(graycoprops(glcm, 'correlation'))
    
    return [features['contrast'], features['dissimilarity'], features['homogeneity'], 
            features['ASM'], features['energy'], features['correlation']]


# Load Data

In [4]:
labels_df = pd.read_excel(r'//home//jaykishor_c//ml//gt_WIndow .xlsx')  

image_folder = r'//home//jaykishor_c//ml//color_window_double1//color_window_double1'  

# GLCM feature sequences for each character across all images

In [5]:
character_glcm_sequences = {}

window_width = 80  # Width of each sliding window in pixels
step_size = 20     # Step size of the sliding window in pixels

# Process each image (word) in the dataset
for index, row in labels_df.iterrows():
    image_name = row['image name']          # image name of respective gt text
    character_sequence = row['gt']          # gt text
    
    # Load the corresponding image
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image {image_name} could not be loaded.")
        continue

    image_width = image.shape[1]  # Get image width
    
    # Calculate the width of each character region based on the sequence length
    num_characters = len(character_sequence)
    character_width = image_width // num_characters
    
    # Loop through each character in the sequence and collect its GLCM features
    for i, char in enumerate(character_sequence):
        # Define the region corresponding to the current character
        region_start = i * character_width
        region_end = region_start + character_width
        character_region = image[:, region_start:region_end]  # Assume height is all rows
        
        # Split the character region into windows to capture GLCM features
        num_windows = (character_width - window_width) // step_size + 1
        char_glcm_sequence = []
        
        for j in range(num_windows):
            # Calculate the start and end of the window within the character region
            window_start = region_start + j * step_size
            window_end = window_start + window_width
            
            # Extract the window
            window = image[:, window_start:window_end]
            
            # Compute GLCM features for this window across four directions
            glcm_features = compute_glcm_features_4_directions(window)
            char_glcm_sequence.append(glcm_features)
        
        # Append this character's GLCM features to the global dictionary
        if char not in character_glcm_sequences:
            character_glcm_sequences[char] = []
        character_glcm_sequences[char].append(char_glcm_sequence)

# Print the GLCM features for each character

In [6]:
# Print the number of GLCM feature sequences for each character
for char, sequences in character_glcm_sequences.items():
    print(f"Character '{char}'  has {len(sequences)}    sequences of GLCM features.")


Character 'െ'  has 618    sequences of GLCM features.
Character 'ത'  has 535    sequences of GLCM features.
Character 'അ'  has 103    sequences of GLCM features.
Character '്'  has 1202    sequences of GLCM features.
Character 'ാ'  has 993    sequences of GLCM features.
Character 'ി'  has 729    sequences of GLCM features.
Character 'പ'  has 339    sequences of GLCM features.
Character 'റ'  has 432    sequences of GLCM features.
Character 'ര'  has 681    sequences of GLCM features.
Character 'ു'  has 455    sequences of GLCM features.
Character 'ള'  has 78    sequences of GLCM features.
Character 'ക'  has 473    sequences of GLCM features.
Character 'ഷ'  has 105    sequences of GLCM features.
Character 'മ'  has 361    sequences of GLCM features.
Character 'ആ'  has 28    sequences of GLCM features.
Character 'ഠ'  has 61    sequences of GLCM features.
Character 'ൽ'  has 40    sequences of GLCM features.
Character 'ർ'  has 62    sequences of GLCM features.
Character 'ഇ'  has 30    sequenc

# Save the GLCM features to a file for analysis


In [7]:
output_path = r'//home//jaykishor_c//ml//Output_glcmcharacter_glcm_features.xlsx'
features_df = pd.DataFrame({
    'Character': [char for char, sequences in character_glcm_sequences.items() for _ in sequences],
    'GLCM Features': [seq for sequences in character_glcm_sequences.values() for seq in sequences]
})



features_df.to_excel(output_path, index=False)
print(f"GLCM features saved to {output_path}")

GLCM features saved to //home//jaykishor_c//ml//Output_glcmcharacter_glcm_features.xlsx


In [8]:
len(char_glcm_sequence)

5

# Storing Character HMMs

In [9]:
character_hmms = {}
num_states = 4

# Example: 'character_glcm_sequences' contains the GLCM feature sequences for each character
for char, sequences in character_glcm_sequences.items():
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    
    if len(sequences) == 0:
        print(f"Warning: No valid sequences for character {char}. Skipping this character.")
        continue  # Skip to the next character if no valid sequences
    
    if len(sequences) >= 15:
        # Prepare training data for the HMM
        X = np.vstack(sequences)  # Stack the sequences into a single array
        lengths = [len(seq) for seq in sequences]  # Length of each sequence

        # Initialize HMM for this character
        model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=1000, init_params='', verbose=True)


        # Use the custom model
        model.fit(X, lengths)

        # Store the trained model
        character_hmms[char] = model
        print(f"Model trained for character: {char}")

         1   -4587.43115380             +nan
         2    3891.91938930   +8479.35054311
         3    5676.70650659   +1784.78711729
         4    7417.02708845   +1740.32058186
         5    8318.67333970    +901.64625125
         6    8731.11176402    +412.43842432
         7    8900.01813991    +168.90637589
         8    9152.12830970    +252.11016979
         9    9277.66627845    +125.53796875
        10    9294.09648458     +16.43020613
        11    9295.72155872      +1.62507414
        12    9296.24156406      +0.52000534
        13    9296.14572055      -0.09584351
Model is not converging.  Current: 9296.145720549604 is not greater than 9296.241564060752. Delta is -0.09584351114790479
         1      -7.75532017             +nan
         2    2814.75315940   +2822.50847957
         3    3077.67212462    +262.91896521
         4    3107.47060906     +29.79848444
         5    3117.16201164      +9.69140258
         6    3127.17620682     +10.01419518
         7    3140.7953

Model trained for character: െ


        11    3152.31609812      +1.51750823
        12    3153.55252013      +1.23642201
        13    3153.95071215      +0.39819202
        14    3154.11871363      +0.16800148
        15    3154.22187888      +0.10316525
        16    3154.29652981      +0.07465093
        17    3154.35300976      +0.05647995
        18    3154.39608385      +0.04307409
        19    3154.42899596      +0.03291210
        20    3154.45430585      +0.02530990
        21    3154.47431136      +0.02000551
        22    3154.49193050      +0.01761914
        23    3154.51347804      +0.02154754
        24    3154.55818485      +0.04470681
        25    3154.69034827      +0.13216343
        26    3155.07946298      +0.38911470
        27    3155.69850973      +0.61904675
        28    3155.98719501      +0.28868528
        29    3156.09994830      +0.11275329
        30    3156.18178153      +0.08183323
        31    3156.24731375      +0.06553222
        32    3156.30152578      +0.05421203
        33

Model trained for character: ത
Model trained for character: അ



         5    1122.44897997    +273.56214038
         6    1379.33087475    +256.88189477
         7    1479.88975706    +100.55888232
         8    1502.88144603     +22.99168897
         9    1506.03371235      +3.15226632
        10    1508.03517988      +2.00146753
        11    1508.80443364      +0.76925376
        12    1512.34476225      +3.54032861
        13    1517.05754557      +4.71278332
        14    1517.15227107      +0.09472550
        15    1517.11496621      -0.03730485
Model is not converging.  Current: 1517.114966213872 is not greater than 1517.1522710668607. Delta is -0.0373048529886546
         1   -4460.25827808             +nan
         2    3419.20799674   +7879.46627483


...

         5     514.04266095      +0.03740376
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
         6     514.04276469      +0.00010375
         1      19.56302025             +nan
         2     467.14100042    +447.57798017
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
         3     495.11857552     +27.97757510
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
         4     562.53571897     +67.41714346
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
         5     628.79648952     +66.26077055
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
         6     667.30280718     +38.50631766
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
         7     679.17162441     +11.86881723
Some rows of transmat_ have zero sum 

Model trained for character: ഭ
Model trained for character: ശ
Model trained for character: ൈ


        19     319.21856974      -0.00402571
Model is not converging.  Current: 319.2185697416661 is not greater than 319.22259544886583. Delta is -0.004025707199730277
         1     -39.24235553             +nan
         2     121.01585306    +160.25820858
         3     159.51507346     +38.49922040
         4     175.36388592     +15.84881246
         5     186.16068865     +10.79680274
         6     190.33303494      +4.17234628
         7     190.58477983      +0.25174489
         8     190.58484676      +0.00006693
         1      93.77897922             +nan
         2     289.47109739    +195.69211817
         3     334.89093804     +45.41984066
         4     350.99192707     +16.10098902
         5     354.12926520      +3.13733813
         6     354.21310527      +0.08384007
         7     354.21747147      +0.00436620
         1      20.74277543             +nan
         2      85.79609583     +65.05332040
         3     100.69999543     +14.90389960
         4     104.07

Model trained for character: ൾ
Model trained for character: ൃ
Model trained for character: ൊ
Model trained for character: ബ


        12     104.40365201      +0.02671383
        13     104.41966703      +0.01601501
        14     104.42818795      +0.00852092


# View the transition matrix after training

In [10]:
print(model.transmat_)  
row_sums = model.transmat_.sum(axis=1)
print(row_sums)

[[1.00000000e+000 9.54760289e-147 4.71485159e-041 1.39449280e-013]
 [4.08565416e-017 7.05685015e-095 1.00000000e+000 2.28847920e-074]
 [2.16447719e-043 1.26047577e-049 1.00000000e+000 1.25071752e-093]
 [3.75800758e-059 3.16517393e-214 2.51543370e-097 1.00000000e+000]]
[1. 1. 1. 1.]


# Save the trained HMM model to a file with the sanitized name

In [11]:
from joblib import dump

# Assuming your models are in a dictionary called `character_hmms`
# e.g., `character_hmms = {'െ': trained_model_1, 'ത': trained_model_2, ... }

def sanitize_filename(char):
    # Replace invalid filename characters with an underscore or remove them
    return re.sub(r'[<>:"/\\|?*]', '_', char)

# Ensure the directory for the models exists
model_directory = '//home//jaykishor_c//ml//Models_glcm'
os.makedirs(model_directory, exist_ok=True)

for char, model in character_hmms.items():
    # Sanitize character to create a valid filename
    sanitized_char = sanitize_filename(char)
    print(char, sanitized_char)
    # Save the trained HMM model to a file with the sanitized name in the 'model' directory
    model_path = os.path.join(model_directory, f"{sanitized_char}_hmm.pkl")
    dump(model, model_path)


െ െ
ത ത
അ അ
് ്
ാ ാ
ി ി
പ പ
റ റ
ര ര
ു ു
ള ള
ക ക
ഷ ഷ
മ മ
ആ ആ
ഠ ഠ
ൽ ൽ
ർ ർ
ഇ ഇ
ങ ങ
ന ന
ജ ജ
ട ട
വ വ
ഹ ഹ
ം ം
യ യ
" _
ധ ധ
ച ച
എ എ
ല ല
ഴ ഴ
ദ ദ
ഗ ഗ
ഉ ഉ
ണ ണ
ൂ ൂ
ൻ ൻ
സ സ
ീ ീ
ഭ ഭ
ശ ശ
ൈ ൈ
ൾ ൾ
ൃ ൃ
ൊ ൊ
ബ ബ


In [12]:
print(X.shape)  

(26, 6)


# Testing shell 1

In [13]:
import joblib
from hmmlearn import hmm
import cv2
import numpy as np
import pandas as pd
 
 
# Function to compute the actual sequence based on window mapping
# Function to extract the actual sequence for a specific image from the DataFrame
def get_actual_sequence_from_df(image_name, line_image, label_df, window_width=30, step_size=10):
    # Find the corresponding row in the DataFrame
    row = label_df[label_df['image name'] == image_name]
    if row.empty:
        raise ValueError(f"Image name '{image_name}' not found in the labels DataFrame.")
    # Extract the ground truth character sequence
    character_sequence = row['gt'].values[0]  # Adjust column name if necessary
    # Compute the actual sequence based on ground truth and image dimensions
    actual_sequence = []
    image_width = line_image.shape[1]
    num_characters = len(character_sequence)
    character_width = image_width // num_characters
 
    for i, char in enumerate(character_sequence):
        # Define the region corresponding to this character
        region_start = i * character_width
        region_end = region_start + character_width
        character_region = line_image[:, region_start:region_end]
 
        # Divide the character region into windows
        num_windows = (character_width - window_width) // step_size + 1
        for _ in range(num_windows):
            actual_sequence.append(char)  # Map each window to the current character
    return actual_sequence
# Function to predict the sequence based on HMM models
def predict_line_sequence(line_image, window_width=30, step_size=10):
    predictions = []
    image_width = line_image.shape[1]
    num_windows = (image_width - window_width) // step_size + 1
 
    for i in range(num_windows):
        window_start = i * step_size
        window_end = window_start + window_width
        window = line_image[:, window_start:window_end]
 
        glcm_features = compute_glcm_features_4_directions(window)
        glcm_features = np.array(glcm_features)  # Convert list to Numpy array if necessary

        #gabor_features = compute_gabor_features(window)
 
            # Combine features (concatenate DCT and Gabor features)
        #combined_features = np.concatenate((dct_features, gabor_features))
 
        combined_features = glcm_features.reshape(1, -1)  # Reshape for HMM input

 
        # Calculate likelihoods for each character model
        best_char = None
        best_score = float('-inf')
 
        for char, model in character_hmms.items():
            try:
                score = model.score(combined_features)
                if score > best_score:
                    best_score = score
                    best_char = char
            except:
                pass  # Ignore errors for invalid model scoring
 
        if best_char is not None:
            predictions.append(best_char)
 
    return ''.join(predictions)
 
# Test image details
# Load the labels DataFrame
label_file = r"//home//jaykishor_c//ml//gt_WIndow .xlsx"
label_df = pd.read_excel(label_file)
 
# Test image details
test_image_path = r"//home//jaykishor_c//ml//color_window_double1//color_window_double1//MaI14_007_3.jpg_window_2.jpg"
test_image_name = test_image_path.split("/")[-1]  # Correctly extracts the image name by splitting on '/'

# print("test image name : ", test_image_name)
# print("Data frame image name : ", label_df['image name'])
 
# Load the test image
test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
 
# Extract the actual sequence from the DataFrame
actual_sequence = get_actual_sequence_from_df(test_image_name, test_image, label_df)
 
# Predict the sequence using the HMM models
predicted_sequence = predict_line_sequence(test_image)
 
# Print the sequences for comparison
print(f"Actual sequence: {''.join(actual_sequence)}")
print(f"Predicted sequence: {predicted_sequence}")

Actual sequence: വവവവവവവവൃൃൃൃൃൃൃൃഥഥഥഥഥഥഥഥ
Predicted sequence: ീൈ""""""""""ൈയയൈൈൈൈൈ""""""ൈൈൈൈ


# Extract the character names from the character_hmms 

In [14]:
char_names = list(character_hmms.keys())

# Print the character names
print("Character names:", char_names)

Character names: ['െ', 'ത', 'അ', '്', 'ാ', 'ി', 'പ', 'റ', 'ര', 'ു', 'ള', 'ക', 'ഷ', 'മ', 'ആ', 'ഠ', 'ൽ', 'ർ', 'ഇ', 'ങ', 'ന', 'ജ', 'ട', 'വ', 'ഹ', 'ം', 'യ', '"', 'ധ', 'ച', 'എ', 'ല', 'ഴ', 'ദ', 'ഗ', 'ഉ', 'ണ', 'ൂ', 'ൻ', 'സ', 'ീ', 'ഭ', 'ശ', 'ൈ', 'ൾ', 'ൃ', 'ൊ', 'ബ']


In [15]:
len(char_names)

48

In [16]:
ground_truth =char_names

In [17]:
print("Ground truth list:", ground_truth)

Ground truth list: ['െ', 'ത', 'അ', '്', 'ാ', 'ി', 'പ', 'റ', 'ര', 'ു', 'ള', 'ക', 'ഷ', 'മ', 'ആ', 'ഠ', 'ൽ', 'ർ', 'ഇ', 'ങ', 'ന', 'ജ', 'ട', 'വ', 'ഹ', 'ം', 'യ', '"', 'ധ', 'ച', 'എ', 'ല', 'ഴ', 'ദ', 'ഗ', 'ഉ', 'ണ', 'ൂ', 'ൻ', 'സ', 'ീ', 'ഭ', 'ശ', 'ൈ', 'ൾ', 'ൃ', 'ൊ', 'ബ']


# Converting 'character to state' and 'state to character'

In [18]:
unique_chars = sorted(set("".join(ground_truth)))
char_to_state = {char: idx for idx, char in enumerate(unique_chars)}
state_to_char = {idx: char for char, idx in char_to_state.items()}

# Testing shel 2

In [ ]:
import os
import cv2
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
 
# Function to calculate BLEU score
def calculate_bleu_score(actual_sequence, predicted_sequence):
    reference = [list(actual_sequence)]  # Wrap in another list for multiple references
    hypothesis = list(predicted_sequence)
    bleu_score = sentence_bleu(reference, hypothesis)
    return bleu_score
 
# Function to calculate accuracy
def calculate_accuracy(actual_sequence, predicted_sequence):
    # Ensure the sequences are the same length for comparison
    min_length = min(len(actual_sequence), len(predicted_sequence))
    if min_length == 0:
        return 0  # Return 0 if either sequence is empty to avoid division by zero
    correct_predictions = sum(1 for i in range(min_length) if actual_sequence[i] == predicted_sequence[i])
    accuracy = (correct_predictions / min_length) * 100  # Calculate accuracy as a percentage
    return accuracy

 
# Folder paths and files
image_folder = r"//home//jaykishor_c//ml//color_window_double1//color_window_double1"
label_file = r"//home//jaykishor_c//ml//gt_WIndow .xlsx"
output_file = r"//home//jaykishor_c//ml//Output_glcm//BLEU_Scores_Accuracy_actual_and_predicated_char.xlsx"
 
# Load the labels DataFrame
label_df = pd.read_excel(label_file)
 
# List all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]
 
# Initialize a results list and variable to track total accuracy
results = []
total_accuracy = 0
num_images = 0
 
# Process each image
for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error loading image: {image_name}")
        continue
 
    # Extract the actual sequence from the DataFrame
    actual_sequence = get_actual_sequence_from_df(image_name, image, label_df)
    if actual_sequence is None:
        print(f"No label found for image: {image_name}")
        continue
 
    # Predict the sequence using the HMM models
    predicted_sequence = predict_line_sequence(image)
    # Calculate BLEU score
    # bleu_score = calculate_bleu_score(actual_sequence, predicted_sequence)
    # Calculate accuracy
    accuracy = calculate_accuracy(actual_sequence, predicted_sequence)
    print(image_name,accuracy)
    # Add the accuracy to the total
    total_accuracy += accuracy
    num_images += 1
    # Append results
    results.append({
        "Image Name": image_name,
        "Actual Sequence": ''.join(actual_sequence),
        "Predicted Sequence": predicted_sequence,
        # "BLEU Score": bleu_score,
        "Accuracy": accuracy
    })
 
# Calculate the average accuracy
average_accuracy = total_accuracy / num_images if num_images > 0 else 0
 
# Save the results to an Excel sheet
results_df = pd.DataFrame(results)
results_df.to_excel(output_file, index=False)
 
# Print average accuracy
print(f"Average Accuracy: {average_accuracy:.2f}%")
print(f"Results saved to {output_file}")

MaI14_007_4.jpg_window_16.jpg 75.0
MaI14_007_9.jpg_window_31.jpg 0.0
MaI14_051_06.jpg_window_27.jpg 0.0
MaI14_051_07.jpg_window_16.jpg 0.0


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


maI16_01_8.jpg_window_5.jpg 0.0
MaI12_Page102_line_3.jpg_window_32.jpg 0.0
MaI849_041_line_5.jpg_window_51.jpg 0.0


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


MaI849_038_line_5.jpg_window_23.jpg 12.5
MaI14_051_04.jpg_window_40.jpg 71.42857142857143


...

MaI849_041_line_4.jpg_window_6.jpg 12.5


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


MaI849_041_line_7.jpg_window_18.jpg 16.666666666666664
MaI14_007_8.jpg_window_14.jpg 4.166666666666666
MaI849_039_line_4.jpg_window_19.jpg 8.333333333333332
MaI849_039_line_8.jpg_window_11.jpg 0.0
MaI849_038_line_4.jpg_window_15.jpg 0.0


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

MaI849_039_line_9.jpg_window_18.jpg 0.0
MaI12_Page101_line_1.jpg_window_29.jpg 0.0
MaI12_Page100_line_4.jpg_window_5.jpg 0.0
Average Accuracy: 7.81%
Results saved to //home//jaykishor_c//ml//Output_glcm//BLEU_Scores_Accuracy_actual_and_predicated_char.xlsx


# Showing the result

In [20]:
results_df

,Image Name,Actual Sequence,Predicted Sequence,BLEU Score,Accuracy
0,MaI14_007_4.jpg_window_16.jpg,"""""""""""""""""""""""""""""""""""""""""""""""""""""""""","""""""""""""""""""""""""""""""""""""""""""ൈരരരയയയയാ",6.836103e-01,75.0
1,MaI14_007_9.jpg_window_31.jpg,ൃൃൃൃൃൃൃൃൃൃൃൃൃൃധധധധധധധധധധധധധധ,"ൈവ""""""വവരരൈൈൈൈീീീീീവവവീീവ""വരവവവ",0.000000e+00,0.0
2,MaI14_051_06.jpg_window_27.jpg,ശശശ്്്ചചചകകക്്്രരര,"""""""""""ാീൈൈൈൈൈൈീീീീീീൈൈൈതൈതൈതത""ത",0.000000e+00,0.0
3,MaI14_051_07.jpg_window_16.jpg,ഥഥഥഥഥഥഥഥഥഥഥഥഥഥിിിിിിിിിിിിിി,"ംം""""""""രീീൈൈൈൈൈദീീവരരരരവീൈൈരരരര",0.000000e+00,0.0
4,maI16_01_8.jpg_window_5.jpg,ാാനനിിയയതത്്സസ,"ചയൈൈൊൈൊൊളൈൈൈയയയയളാാ""ാ""ാാാാ""""""""",4.619180e-155,0.0
...,...,...,...,...,...
3531,MaI849_039_line_8.jpg_window_11.jpg,സസസസസസസസസസസസസസഭഭഭഭഭഭഭഭഭഭഭഭഭഭ,ളൾഎഎഎതതൾൾൾൾഎഎൊംംംരരൾെ്്്ഇെൾരറ,0.000000e+00,0.0
3532,MaI849_038_line_4.jpg_window_15.jpg,രരരരരരരരരരരരരരുുുുുുുുുുുുുു,"""""""""""""""""""""""""""ാാാാ""ാവവവ""""""""""""""വ",0.000000e+00,0.0
3533,MaI849_039_line_9.jpg_window_18.jpg,നനനനനന്്്്്്നനനനനനഅഅഅഅഅഅ,"പംററററററററ""""""""""ദണററണറപപററററററഅ",7.784451e-232,0.0
3534,MaI12_Page101_line_1.jpg_window_29.jpg,നനനനനന്്്്്്നനനനനനുുുുുു,ആആആആമആആആആആധആആആആആഇഇജങഇആഉധധധധധധധ,0.000000e+00,0.0


# BLUE score for each character

In [21]:
from nltk.translate.bleu_score import sentence_bleu

characters_to_remove = ['ണ',' ൾ','ു','ഥ','ൃ','ബ','ൂഃ','ഃ','ൻ']
 
# Calculate BLEU score for predictions
for char, model in character_hmms.items():
    try:
        sequences = character_glcm_sequences[char]
        # Remove empty sequences
        sequences = [seq for seq in sequences if len(seq) > 0]
        # Check if we have any sequences left
        if len(sequences) >= 15:
            X = np.vstack(sequences)
            lengths = [len(seq) for seq in sequences]
            # Check for valid transition matrix
            row_sums = model.transmat_.sum(axis=1)
            if not np.allclose(row_sums, 1):
                print(f"Problem with transition matrix for character '{char}': row sums = {row_sums}")
                characters_to_remove.append(char)
                continue  # Skip this model and move to the next one
            # Get predicted states
            predicted_states = model.predict(X)
            mapped_predictions = [state_to_char[state] for state in predicted_states]
            # Calculate BLEU score
            bleu_score = sentence_bleu([ground_truth], mapped_predictions)
            print(f"BLEU Score for character '{char}': {bleu_score}")
    except ValueError as e:
        print(f"Error with character '{char}': {e}")

BLEU Score for character 'െ': 3.7543342543670055e-232
BLEU Score for character 'ത': 4.9200425788916696e-232
BLEU Score for character 'അ': 5.4269696562331364e-232
BLEU Score for character '്': 5.0806376019353144e-232
BLEU Score for character 'ാ': 3.961047620822523e-232
BLEU Score for character 'ി': 4.614309862807827e-232
BLEU Score for character 'പ': 5.2365941639833255e-232
BLEU Score for character 'റ': 4.879980276268283e-232
BLEU Score for character 'ര': 4.4943550935417704e-232
BLEU Score for character 'ു': 5.569381294996544e-232
BLEU Score for character 'ള': 7.063663544433296e-232
BLEU Score for character 'ക': 4.902213108167789e-232
Problem with transition matrix for character 'ഷ': row sums = [1. 1. 0. 1.]
BLEU Score for character 'മ': 5.165360761805455e-232
BLEU Score for character 'ആ': 6.94876893293765e-232
BLEU Score for character 'ഠ': 6.851187014592609e-232
BLEU Score for character 'ൽ': 7.50359907734765e-232
BLEU Score for character 'ർ': 7.420480236834311e-232
BLEU Score for chara

/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

# Remove characters if they exist in the dictionary

In [22]:
for char in characters_to_remove:
    print (char)
    if char in character_hmms:
        del character_hmms[char]
        print(f"Deleted HMM model for character: {char}")
    else:
        print(f"Character {char} not found in character_hmms.")

ണ
Deleted HMM model for character: ണ
 ൾ
Character  ൾ not found in character_hmms.
ു
Deleted HMM model for character: ു
ഥ
Character ഥ not found in character_hmms.
ൃ
Deleted HMM model for character: ൃ
ബ
Deleted HMM model for character: ബ
ൂഃ
Character ൂഃ not found in character_hmms.
ഃ
Character ഃ not found in character_hmms.
ൻ
Deleted HMM model for character: ൻ
ഷ
Deleted HMM model for character: ഷ
ട
Deleted HMM model for character: ട
സ
Deleted HMM model for character: സ
ഭ
Deleted HMM model for character: ഭ
ശ
Deleted HMM model for character: ശ


# Calculate BLEU score for predictions

In [23]:
# Calculate BLEU score for predictions (this is more useful for sequence generation tasks)
for char, model in character_hmms.items():
    sequences = character_glcm_sequences[char]
    
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    
    # Check if we have any sequences left
    if len(sequences) >=15:
       
        X = np.vstack(sequences)
    
    lengths = [len(seq) for seq in sequences]
    
    # Get predicted states
    predicted_states = model.predict(X)
    mapped_predictions = [state_to_char[state] for state in predicted_states]

# Calculate BLEU score
bleu_score = sentence_bleu([ground_truth], mapped_predictions)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 8.094051362019983e-232


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

# Save the BLUR score to an Excel file

In [24]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

# List to store results
results = []

# Loop through each character and its model
for char, model in character_hmms.items():
    sequences = character_glcm_sequences[char]
    
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    
    # Check if we have enough sequences to proceed
    if len(sequences) >= 15:
        X = np.vstack(sequences)

        lengths = [len(seq) for seq in sequences]
        
        # Get predicted states and map to characters
        predicted_states = model.predict(X)
        mapped_predictions = [state_to_char[state] for state in predicted_states]
        
        # Calculate BLEU score
        bleu_score = sentence_bleu([ground_truth], mapped_predictions)
        
        # Store the results
        results.append({
            'Character': char,
            'Mapped Predictions': ''.join(mapped_predictions),
            'BLEU Score': bleu_score
        })

# Create a DataFrame to store results
results_df = pd.DataFrame(results)


# Save the results to an Excel file
output_path = r'//home//jaykishor_c//ml//Output_glcm//mapped_predictions_bleu_scores.xlsx'
results_df.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")

/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

Results saved to //home//jaykishor_c//ml//Output_glcm//mapped_predictions_bleu_scores.xlsx


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

In [25]:
results_df

,Character,Mapped Predictions,BLEU Score
0,െ,ആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആ...,3.754334e-232
1,ത,അഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅ...,4.920043e-232
2,അ,"ആഅആഅആഅആഅഅആഅആഅആ""""""""""ആഅആഅആഅആഅആഅആഅആഅആഅആ""""""ആഅആഅആഅആ...",5.426970e-232
3,്,ആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആ...,5.080638e-232
4,ാ,"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",3.961048e-232
5,ി,"""""""ആആആആആആആആആആ""""""""""""""""""""""""""""ആആആആആആ""""""""""""""""""""""""""...",4.614310e-232
6,പ,"""""""""""""""""അഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅ""""...",5.236594e-232
7,റ,"ആആആആആആആആആആആആആആആ""""""""""""""""""""ആആആ""""""""ആആആആആആആആആആആ""അം...",4.879980e-232
8,ര,"""""""""""""""""""""""""ംംംംംംംംം""""""""""ം""""""""""""ംംംംംംംംംംംംം...",4.494355e-232
9,ള,"ംംംംംംംംംംംംംംം""ആആംംംംംംംം""ആആആആആ""അഅഅഅഅഅഅഅഅഅഅഅഅ...",7.063664e-232


# Calculate Probability

**1. Log Probability**

In [26]:
'''for char, model in character_hmms.items():
    sequences = character_glcm_sequences[char]
    sequences = [seq for seq in sequences if len(seq) > 0]

    if len(sequences) >= 15:
        X = np.vstack(sequences)
        lengths = [len(seq) for seq in sequences]
        log_probability = model.score(X, lengths)
        print(f"Log Probability for character '{char}': {log_probability}")
'''

'for char, model in character_hmms.items():\n    sequences = character_glcm_sequences[char]\n    sequences = [seq for seq in sequences if len(seq) > 0]\n\n    if len(sequences) >= 15:\n        X = np.vstack(sequences)\n        lengths = [len(seq) for seq in sequences]\n        log_probability = model.score(X, lengths)\n        print(f"Log Probability for character \'{char}\': {log_probability}")\n'

**2. Model Probability**

In [27]:
'''for char, model in character_hmms.items():
    sequences = character_glcm_sequences[char]
    sequences = [seq for seq in sequences if len(seq) > 0]

    if len(sequences) >= 15:
        X = np.vstack(sequences)
        lengths = [len(seq) for seq in sequences]
        log_probability = model.score(X, lengths)
        model_probability = np.exp(log_probability)  # Convert to regular probability
        print(f"Model Probability for character '{char}': {model_probability}")
'''

'for char, model in character_hmms.items():\n    sequences = character_glcm_sequences[char]\n    sequences = [seq for seq in sequences if len(seq) > 0]\n\n    if len(sequences) >= 15:\n        X = np.vstack(sequences)\n        lengths = [len(seq) for seq in sequences]\n        log_probability = model.score(X, lengths)\n        model_probability = np.exp(log_probability)  # Convert to regular probability\n        print(f"Model Probability for character \'{char}\': {model_probability}")\n'

**3. Transition Matrix Validation**

In [28]:
'''for char, model in character_hmms.items():
    row_sums = model.transmat_.sum(axis=1)
    if not np.allclose(row_sums, 1):
        print(f"Transition matrix issue for '{char}': {row_sums}")
    else:
        print(f"Transition matrix for '{char}' is valid.")
'''

'for char, model in character_hmms.items():\n    row_sums = model.transmat_.sum(axis=1)\n    if not np.allclose(row_sums, 1):\n        print(f"Transition matrix issue for \'{char}\': {row_sums}")\n    else:\n        print(f"Transition matrix for \'{char}\' is valid.")\n'

**4. Emission Probabilities**

In [29]:
'''for char, model in character_hmms.items():
    print(f"Emission means for character '{char}': {model.means_}")
    print(f"Emission covariances for character '{char}': {model.covars_}")
'''

'for char, model in character_hmms.items():\n    print(f"Emission means for character \'{char}\': {model.means_}")\n    print(f"Emission covariances for character \'{char}\': {model.covars_}")\n'

**5. Per-State Log Probability**

In [30]:
'''for char, model in character_hmms.items():
    sequences = character_glcm_sequences[char]
    sequences = [seq for seq in sequences if len(seq) > 0]

    if len(sequences) >= 15:
        X = np.vstack(sequences)  # Combine all sequences into a single array

        # Use score_samples to compute posterior probabilities for states
        log_prob, posteriors = model.score_samples(X)
        print(f"Log probabilities for character '{char}': {log_prob}")
        print(f"Posterior probabilities for states:\n{posteriors}")
'''

'for char, model in character_hmms.items():\n    sequences = character_glcm_sequences[char]\n    sequences = [seq for seq in sequences if len(seq) > 0]\n\n    if len(sequences) >= 15:\n        X = np.vstack(sequences)  # Combine all sequences into a single array\n\n        # Use score_samples to compute posterior probabilities for states\n        log_prob, posteriors = model.score_samples(X)\n        print(f"Log probabilities for character \'{char}\': {log_prob}")\n        print(f"Posterior probabilities for states:\n{posteriors}")\n'

# Actual and Prediction of character

In [31]:
# Dictionary to store actual vs predicted characters
results = {'Actual': [], 'Predicted': []}
 
# Process each test character and its sequences
for actual_char, test_sequences in character_glcm_sequences.items():
    for test_sequence in test_sequences:
        test_sequence = np.array(test_sequence)
        # Dictionary to store scores for each character model
        scores = {}
        # Score the test sequence against each character's HMM model
        for char, model in character_hmms.items():
            try:
                scores[char] = model.score(test_sequence)
            except:
                scores[char] = -np.inf  # If model can't score, assign a low score
        # Predict the character with the highest score
        predicted_char = max(scores, key=scores.get)
        # Store the result
        results['Actual'].append(actual_char)
        results['Predicted'].append(predicted_char)
 
# Save the results to an Excel file
output_path = r'/home//jaykishor_c//ml//Output_glcm//actual_vs_predicted.xlsx'
results_df = pd.DataFrame(results)
results_df.to_excel(output_path, index=False)


print(f"Results saved to {output_path}")

Results saved to /home//jaykishor_c//ml//Output_glcm//actual_vs_predicted.xlsx


# Calculate Accuracy

In [32]:

correct_predictions = sum([1 for actual, predicted in zip(results['Actual'], results['Predicted']) if actual == predicted])
total_predictions = len(results['Actual'])
accuracy = (correct_predictions / total_predictions) * 100

# Print accuracy
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 6.74%
